In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
history = pd.read_csv("/Users/zhouxueqi/Desktop/history.csv")
predict = pd.read_csv("/Users/zhouxueqi/Desktop/Fall2020/doordash/predict_data.csv")
feature = pd.read_csv("/Users/zhouxueqi/Desktop/combine_train_test.csv")

In [3]:
train_data = feature[:197428]
test_data = feature[197428:]

In [4]:
train_data_Y = history['target']
train_data_X = train_data
test_data_X = test_data
X = train_data_X
y = train_data_Y

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [6]:
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor#导入模型

In [7]:
def rmse_cv(model,X_train,y_train):#输入模型，X，Y，输出RMSE
    rmse = np.sqrt(-cross_val_score(model, X_train,y_train, scoring="neg_mean_squared_error", cv=5))
    return rmse

In [8]:
# rmse value for every single model
models = [LinearRegression(),Ridge(),Lasso(alpha=0.01,max_iter=10000),RandomForestRegressor(),GradientBoostingRegressor(),LinearSVR(),
          ElasticNet(alpha=0.001,max_iter=10000),BayesianRidge(),ExtraTreesRegressor(),XGBRegressor()]
names = ["LR", "Ridge", "Lasso", "RF", "GBR", "LinSVR", "Ela","Bay","Extra","Xgb"]
for name, model in zip(names, models):
    score = rmse_cv(model, X_train,y_train)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

In [9]:
# the model I finnaly choose is Lasso, GBR, Ela, Bay, Extra, Xgb
# find the best parameter

In [25]:
## define cross validation to search for parameter
## CV=5
# class grid():
#     def __init__(self,model):
#         self.model = model
    
#     def grid_get(self,X,y,param_grid):
#         grid_search = GridSearchCV(self.model,param_grid,cv=5, scoring="neg_mean_squared_error")
#         grid_search.fit(X,y)
#         print(grid_search.best_params_, np.sqrt(-grid_search.best_score_))
#         grid_search.cv_results_['mean_test_score'] = np.sqrt(-grid_search.cv_results_['mean_test_score'])
#         print(pd.DataFrame(grid_search.cv_results_)[['params','mean_test_score','std_test_score']])

In [26]:
# try to find the best parameters for single model
# grid(Lasso()).grid_get(X,y,{'alpha': [0.005,0.01,0.02,0.03],'max_iter':[10000]})

{'alpha': 0.03, 'max_iter': 10000} 1063.8731689552897
                                params  mean_test_score  std_test_score
0  {'alpha': 0.005, 'max_iter': 10000}      1064.136538    66072.288606
1   {'alpha': 0.01, 'max_iter': 10000}      1064.038628    66072.980947
2   {'alpha': 0.02, 'max_iter': 10000}      1063.930912    66196.698816
3   {'alpha': 0.03, 'max_iter': 10000}      1063.873169    66355.802372


In [12]:

# grid(GradientBoostingRegressor()).grid_get(X,y,{'n_estimators':[10,100,1000,10000],'alpha':[0.1,0.3,0.5,0.8]})

In [13]:
# grid(ElasticNet()).grid_get(X,y,{'alpha':[0.0005,0.0008,0.004,0.005],'l1_ratio':[0.08,0.1,0.3,0.5,0.7],'max_iter':[10000]})

In [14]:
# experience on running the single model
# Xgb = XGBRegressor()
# Xgb.fit(X_train, y_train)
# y_pred = Xgb.predict(X_test)

In [15]:
# from sklearn.metrics import mean_squared_error
# from math import sqrt
# rms = sqrt(mean_squared_error(y_test, y_pred))
# print(rms)

In [16]:
# y_res = Xgb.predict(test_data_X)
# result=pd.DataFrame({'delivery_id':predict['delivery_id'], 'predicted_duration':y_res})

In [17]:
# result.to_csv("/Users/zhouxueqi/Desktop/data_to_predict.csv",index=False)

In [8]:
from sklearn.base import BaseEstimator
from sklearn.base import RegressorMixin
from sklearn.linear_model import Lasso

In [9]:
## define weighted average class
class AverageWeight(BaseEstimator, RegressorMixin):
    def __init__(self,mod,weight):
        self.mod = mod##模型的个数
        self.weight = weight##权重
        
    def fit(self,X,y):
        self.models_ = [x for x in self.mod]
        for model in self.models_:
            model.fit(X,y)
        return self
    
    def predict(self,X):
        w = list()
        pred = np.array([model.predict(X) for model in self.models_])
        # 针对于每一个数据点，单一的模型是乘以权重，然后加起来
        for data in range(pred.shape[1]):
            single = [pred[model,data]*weight for model,weight in zip(range(pred.shape[0]),self.weight)]
            w.append(np.sum(single))
        return w

#指定每一个算法的参数
Las = Lasso(alpha=0.01,max_iter=10000)
GBR = GradientBoostingRegressor()
Ela = ElasticNet(alpha=0.001,max_iter=10000)
Bay = BayesianRidge()
Extra = ExtraTreesRegressor()
Xgb = XGBRegressor()

##6个权重，自定义的
w1 = 0.1
w2 = 0.2
w3 = 0.1
w4 = 0.1
w5 = 0.2
w6 = 0.3

#调用函数
weight_avg = AverageWeight(mod = [Las, GBR, Ela, Bay, Extra, Xgb],weight=[w1,w2,w3,w4,w5,w6])
##计算出交叉验证的均值
print(rmse_cv(weight_avg,X,y))
print(rmse_cv(weight_avg,X,y).mean())

/Users/zhouxueqi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[00:19:19] WARNING: /usr/local/miniconda/conda-bld/xgboost_1566327371504/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/zhouxueqi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[00:29:51] WARNING: /usr/local/miniconda/conda-bld/xgboost_1566327371504/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/zhouxueqi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

[01:33:44] WARNING: /usr/local/miniconda/conda-bld/xgboost_1566327371504/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/zhouxueqi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:44:46] WARNING: /usr/local/miniconda/conda-bld/xgboost_1566327371504/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/zhouxueqi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:55:27] WARNING: /usr/local/miniconda/conda-bld/xgboost_1566327371504/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1022.8311100243816


In [10]:
weight_avg.fit(X_train, y_train)
y_pred2 = weight_avg.predict(X_test)

/Users/zhouxueqi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[02:21:50] WARNING: /usr/local/miniconda/conda-bld/xgboost_1566327371504/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [11]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rms = sqrt(mean_squared_error(y_test, y_pred2))
print(rms)

968.642520991147


In [12]:
y_res = weight_avg.predict(test_data_X)
result=pd.DataFrame({'delivery_id':predict['delivery_id'], 'predicted_duration':y_res})
result.to_csv("/Users/zhouxueqi/Desktop/data_to_predict.csv",index=False)